# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [2]:
import pyorient

In [3]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [4]:
print(client.db_list())

{{'databases': {'d': 'plocal:/orientdb/databases/d', 'gods': 'memory:gods'}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [ ]:
# create a database
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [6]:
# connect to the database
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [11]:
# create a new vertex
client.command("insert into V set name = 'Zeus', symbol = 'thunder'")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [20]:
# Print all vertices in Gods
all_goods = client.query("select from V")
for god in all_goods:
    print(god.oRecordData)

{'name': 'Zeus', 'symbol': 'thunder'}
{'name': 'Héra', 'symbol': 'tiara'}
{'name': 'Poséidon', 'symbol': 'trident'}
{'name': 'Athena', 'symbol': 'helmet'}
{'name': 'Arès', 'symbol': 'weapons'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [15]:
# Create a new vertex
Liste_of_gods = [
    {"name": "Héra", "symbol": "tiara"},
    {"name": "Poséidon", "symbol": "trident"},
    {"name": "Athena", "symbol": "helmet"},
    {"name": "Arès", "symbol": "weapons"}
]

for god in Liste_of_gods:
    client.command("insert into V set name = '{}', symbol = '{}'".format(god['name'], god['symbol']))

**Q:** Display all vertices with name = `Arès`

In [19]:
# all Arès
resultat_arès = client.command("select * from V where name = 'Arès'")
for e in resultat_arès:
    print(e.oRecordData)

{'name': 'Arès', 'symbol': 'weapons'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [25]:
# Create a new edge from Zeus to Poséidon with the content kind:'siblings'
client.command("create edge from (select from V where name = 'Zeus') to (select from V where name = 'Poséidon') set kind = 'sibling'")


**Q:** Redisplay all vertices, discuss.

In [26]:
# Redisplay all vertices
all_goods = client.command("select from V")
for god in all_goods:
    print(god.oRecordData)

{'out_': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A127908>, 'name': 'Zeus', 'symbol': 'thunder', 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A08C3C8>}
{'name': 'Héra', 'symbol': 'tiara'}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A08C588>, 'name': 'Poséidon', 'symbol': 'trident', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A08C470>}
{'name': 'Athena', 'symbol': 'helmet'}
{'name': 'Arès', 'symbol': 'weapons'}


**Q:** Display all edges. They are contained in the class `E`

In [27]:
# Display all edges containt in the class E
all_edges = client.command("select from E")
for edge in all_edges:
    print(edge.oRecordData)

{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x000001C98A08CB38>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000001C98A08CA58>}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [28]:
# list of edges
edges = [
    {"from": "Zeus", "to": "Héra", "label": "sibling"},
    {"from": "Zeus", "to": "Arès", "label": "father"},
    {"from": "Zeus", "to": "Athena", "label": "father"},
    {"from": "Héra", "to": "Arès", "label": "mother"},
    {"from": "Héra", "to": "Zeus", "label": "sibling"},
    {"from": "Poséidon", "to": "Zeus", "label": "sibling"}
]

# Create edges
for edge in edges:
    from_vertex = edge["from"]
    to_vertex = edge["to"]
    label = edge["label"]
    query = """
    CREATE EDGE {} FROM (SELECT FROM V WHERE name = '{}') TO (SELECT FROM V WHERE name = '{}')
    """.format(label, from_vertex, to_vertex)
    client.command(query)

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [30]:
# all vertices connected from Zeus
query = """
SELECT expand(out()) FROM V WHERE name = 'Zeus'
"""

for vertex in client.command(query):
    print(vertex.oRecordData)

{'name': 'Arès', 'symbol': 'weapons', 'in_father': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0908>, 'in_mother': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0F60>}
{'name': 'Athena', 'symbol': 'helmet', 'in_father': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0EF0>}
{'name': 'Héra', 'symbol': 'tiara', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B06A0>, 'out_mother': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0208>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0978>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0AC8>, 'name': 'Poséidon', 'symbol': 'trident', 'in_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B08D0>, 'out_Sibling': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0390>}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [32]:
# Requête pour afficher tous les sommets qui ont un père
query = "SELECT FROM V WHERE in('father').size() > 0"
result = client.command(query)

# Afficher les résultats
for vertex in result:
    print(vertex.oRecordData)

{'in_father': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A0B0400>, 'name': 'Athena', 'symbol': 'helmet'}
{'in_father': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A013748>, 'name': 'Arès', 'symbol': 'weapons', 'in_mother': <pyorient.otypes.OrientBinaryObject object at 0x000001C98A127630>}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [34]:
# Requête pour afficher tous les sommets qui ont un arc sortant de type 'mother'
query = "SELECT FROM V WHERE @rid IN (SELECT out FROM E WHERE label = 'mother')"
result = client.command(query)

# Afficher les résultats
for vertex in result:
    print(vertex)

**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [35]:
# Requête pour afficher les frères et sœurs de Zeus
query = """
SELECT FROM V WHERE @rid IN (
    SELECT in FROM E WHERE label = 'sibling' AND out = (SELECT @rid FROM V WHERE name = 'Zeus')
)
"""
result = client.command(query)

# Afficher les résultats
for vertex in result:
    print(vertex)

## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [36]:
# create an eCommerce database
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

In [44]:
# all classes in the database eCommerce
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

{{'name': 'OTriggered'},'version':0,'rid':'#-2:0'}
{{'name': 'V'},'version':0,'rid':'#-2:1'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:2'}
{{'name': 'E'},'version':0,'rid':'#-2:3'}
{{'name': 'OUser'},'version':0,'rid':'#-2:4'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:5'}
{{'name': 'Personnes'},'version':0,'rid':'#-2:6'}
{{'name': 'Products'},'version':0,'rid':'#-2:7'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:8'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:9'}
{{'name': 'ORole'},'version':0,'rid':'#-2:10'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:11'}


In [43]:
# create a new class products
client.command("CREATE CLASS Personnes extends V")
client.command("CREATE CLASS Products extends V")

[12]

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [ ]:
# create the following products
products = [
    {"name": "spaghetti"},
    {"name": "bolognese",},
    {"name": "sauce"},
    {"name": "cheese"},
    {"name": "apple"},
]



**Q:** Create the following people: `peter`, `meredith`.

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


**Q:** Who bought Bolognese sauce?

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.